# **Layer Cake Model**
This is a notebook the details the development of a landscape evolution model that simulates erosion into layers with different erodibilities. Unlike previous research on this topic, the layers will have complex geometries like buried drainage networks. The main equation of interest is the stream power incision model (SPIM).

$E = K A^m S^n$

where $E$ is erosion, $K$ is rock erodibility, $A$ is drainage area, $S$ is slope, and $m$ and $n$ are positive exponents. The first thing we will develop is a simple landscape evolution model using SPIM.

The governing equation is
$\frac{\partial \eta}{\partial t} = \upsilon - K A^m S^n$
where $\eta$ is elevation, $t$ is time, and $\upsilon$ is a rock uplift rate.

## **Landlab**
I will be using landlab to run this model. Let's start by important the libraries we need to run the model.

In [1]:
#Importing the usual suspects
import matplotlib.pyplot as plt
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
%matplotlib inline

#Import landlab components
import warnings
warnings.simplefilter(action='ignore') #stops annoying pandas warning
from landlab import RasterModelGrid #sets up grid
from landlab.components import FlowAccumulator #finds drainage area
from landlab.components import FastscapeEroder #employs SPIM

### **The *Grid***
Now that the libraries are imported we can make the grid. To make a grid, we need to specify how the number of nodes and the size of the nodes. A grid made of 25 rows and 50 columns with a grid size of 100 meters will be 2,500 m x 5,000 m. The grid is typically named $mg$.

In [2]:
rows = 40
columns = 80
dx = 100. #meters

mg = RasterModelGrid((rows,columns),dx) #The Grid

### **Fields**
Now that we have a grid, we can make a field that the grid holds. Since we are tracking landscape evolution, we will need a field that holds elevation values. We name the field 'topographic__elevation' because other built-in functions of landlab expect topography to be named this way. 'eta' holds the array information of 'topographic__elevation'. We can convert 'topographic__elevation' by altering 'eta' or vice versa.

In [3]:
eta = mg.add_zeros('topographic__elevation', at = 'node')

### **Initial Conditions**
We initialize the topographic data with random noise. Let's plot it.

In [4]:
#mg.at_node['topographic__elevation'][mg.core_nodes] = np.random.rand(len(mg.core_nodes)) #For randomized surface
mg.at_node['topographic__elevation'][mg.core_nodes] = 0.0025 * mg.x_of_node[mg.core_nodes] + 0.005 * np.abs(mg.y_of_node[mg.core_nodes] - float(rows*dx)/2.0) + np.random.rand(len(mg.core_nodes)) # for v-shaped valley surface with randominess

from layer_functions import *
jeff_plot(mg,'topographic__elevation',False,'viridis',r'$\eta$ [m]','Initial_Condition_0.png')

### **Boundary Conditions**
We have initial conditions, now we need to set boundary conditions, i.e. rules for what happens at the boundaries. For this model, I want the rivers to only drain out the left boundary. I want the rest of the boundaries to act as walls. This is how we do this in landlab.

In [5]:
#True is closed, False is open
#mg.set_closed_boundaries_at_grid_edges(True, True, False, True)#right, top, left, bottom
jeff_bc_open_or_closed(mg, False, False, True, False)#right_boolean,top_boolean,left_boolean,bottom_boolean)

### **Landlab Functions**
We need to set the landlab functions now. We're going to set up the flow accumulator (finds drainage area) and the fast scale eroder (determines fluvial erosion).

In [6]:
#D8 is the method of finding flow direction
flow = FlowAccumulator(mg, flow_director='D8')
#K_sp is K in the SPIM equation
erode = FastscapeEroder(mg, K_sp=0.00001)

#print all the new fields that these functions generate
print (mg.at_node.keys())

['topographic__elevation', 'water__unit_flux_in', 'drainage_area', 'flow__data_structure_delta', 'flow__upstream_node_order', 'surface_water__discharge', 'flow__sink_flag', 'flow__link_to_receiver_node', 'flow__receiver_node', 'topographic__steepest_slope']


### **Time to Run the Model!**
We've got everything we need. Let's run the model.

In [7]:
dt = 100. #time step, yrs
T = 2. * 10. ** 6. #simulation time, yrs
uplift = 0.001 #uplift rate, m/yr

for t in range(0,int(T/dt)):    
    flow.run_one_step() #find drainage area
    erode.run_one_step(dt=dt) #calculate erosion rate and subtract from the topography
    mg.at_node['topographic__elevation'][mg.core_nodes] += uplift * dt #add uplift
    
    if t%(int(T/dt)/10) == 0: #progress bar
        print (str(round(float(t)/float(T/dt)*100,1)) +'% ', end = '')
print ('done!')   
jeff_plot(mg,'topographic__elevation',False,'viridis',r'$\eta$ [m]','Bedrock_Topography.png')   
jeff_plot(mg,'drainage_area',True,'inferno',r'log(A) [log m$^2$]','Bedrock_Drainage_Area.png')   

0.0% 10.0% 20.0% 30.0% 40.0% 50.0% 60.0% 70.0% 80.0% 90.0% done!


### **Burying the Landscape**
To bury the landscape, we need to create two new fields. One field that signifies the elevation of the buried topography and one that signifies the erodibility of the surface layer. If the erodibilities of the buried material and the burial material are different, we need to tell the model which material it is eroding through by adjusting $K$. Let's assume that the topography is buried 1000 meters, burying even the highest relief in the landscape.

In [8]:
buried_eta = mg.add_zeros('buried__topographic__elevation_1', at = 'node') #buried field
mg.at_node['buried__topographic__elevation_1'][mg.core_nodes] = mg.at_node['topographic__elevation'][mg.core_nodes] - 1000.

### **Using the Lithology Library**
Now, let's assume that the burial material is more erodible by a factor of 10. Next, we re-simulate landscape evolution by setting an initial surface topography again, but this time when the landscape erodes into the basement layer, K will decrease by a factor of 10, slowing down erosion. Let's also change the boundary conditions, so the landscape's outlet is on the right side of the domain.

In [9]:
from landlab.components import Lithology
T = 1.5 * 10. ** 6. #simulation time, yrs

#change boundary condition
jeff_bc_open_or_closed(mg, True, False, False, False)

#reset topography, buriying the landscape
mg.at_node['topographic__elevation'][mg.core_nodes] = np.random.rand(len(mg.core_nodes)) #resetting topography

thicknesses = [mg.at_node['topographic__elevation'] - mg.at_node['buried__topographic__elevation_1'],1000000.]
ids = [1, 2]
attrs = {'K_sp': {1: 0.0001, 2: 0.00001}}
lith = Lithology(mg, thicknesses, ids, attrs, dz_advection = uplift * dt)
lith.run_one_step()
flow.run_one_step() #find drainage area
for t in range(0,int(T/dt)+1):    
    if t%(int(T/dt)/5) == 0: #progress bar
        multirock_plot(mg,attrs['K_sp'][1],t)          
        multirock_profiler(mg,lith,650,t)
        jeff_plot(mg,'drainage_area',True,'inferno',r'log(A) [log m$^2$]','Drainage_Area_'+ '%06d' % t + '.png' )
        print (str(round(float(t)/float(T/dt)*100,1)) +'% ', end = '')
    erode = FastscapeEroder(mg, K_sp=mg.at_node['K_sp'])
    flow.run_one_step() #find drainage area
    erode.run_one_step(dt=dt) #calculate erosion rate and subtract from the topography
    mg.at_node['topographic__elevation'][mg.core_nodes] += uplift * dt #add uplift
    lith.run_one_step()

print ('done!') 

0.0% 20.0% 40.0% 60.0% 80.0% 100.0% done!


To do:
1. Find way to plot multi rocktype DONE
2. Circular Basin
3. Literature review. Why is this important? What's new since the nsf proposal.
4. numerical methods, low diffusion, what happens at interface
5. profile longest stream, or just horizontal?
6. chi vector and rock map


In [1]:
check chi, use multi rock


SyntaxError: invalid syntax (3948878827.py, line 1)